In [1]:
import pandas as pd
from glob import glob
import flow_package as f_p
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split

In [2]:
files = glob("./raw_after_filtered/cicids2017/data/*.csv")
df = pd.DataFrame()

for file in files:
    print(file)
    df_tmp = pd.read_csv(file)
    df = pd.concat([df, df_tmp])
else:
    print(df.shape)


./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_000.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_001.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_002.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_003.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_004.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_005.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_006.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_007.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_008.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_009.csv
./raw_after_filtered/cicids2017/data\Benign-Monday-WorkingHours.pcap_ISCX.csv_010.csv
./raw_after_filtered/cicids2017/data\Botnet-Friday-Wor

In [3]:
df = df.dropna(how="any").dropna(axis=1, how="all")

df = df.reset_index(drop=True).drop(columns=["Unnamed: 0"])
df, ohe = f_p.ohe_hot_encoding(df, categorical_list=["Protocol", "Destination Port"])

print(df["Number Label"].unique())
print(df["Number Label"].value_counts())
length = len(df["Number Label"].unique())

train_data, test_data = train_test_split(df, test_size=0.2, stratify=df["Number Label"], random_state=42)

print(train_data["Number Label"].value_counts())
print(test_data["Number Label"].value_counts())

test_data.to_csv("./test.csv")

MemoryError: Unable to allocate 1.11 TiB for an array with shape (2827876, 53794) and data type float64

## Multi

In [ ]:
value_counts = df["Number Label"].value_counts().sort_index()

value_no2_max = 1000

new_df = pd.DataFrame()
for i in range(length):
    if value_counts[i] > value_no2_max:
        new_df = pd.concat([
            new_df,
            train_data[train_data["Number Label"] == i].sample(value_no2_max, random_state=1),
        ])
    else:
        new_df = pd.concat([
            new_df,
            train_data[train_data["Number Label"] == i]
        ])

In [ ]:
x_df = new_df.drop(columns=["Number Label"])
y_df = new_df["Number Label"]

smote_nc = SMOTENC(
    categorical_features=[x_df.columns.get_loc(label) for label in ohe],
    random_state=42,
    k_neighbors=3,
    sampling_strategy="minority",
)

x_resampled, y_resampled = smote_nc.fit_resample(x_df, y_df)

x_re_df = pd.DataFrame(x_resampled)
y_re_df = pd.DataFrame(y_resampled, columns=["Number Label"])

new_re_df = pd.concat([x_re_df, y_re_df], axis=1)

new_re_df.to_csv("./onehot.csv")

d:\github\drl\.venv\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
d:\github\drl\.venv\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


## Binary

In [ ]:
train_data["Number Label"] = train_data["Number Label"].apply(lambda n_label: 0 if n_label == 0 else 1)
value_counts = train_data["Number Label"].value_counts().sort_index()

value_no2_max = 10000

new_df = pd.DataFrame()
for i in [0, 1]:
    if value_counts[i] > value_no2_max:
        new_df = pd.concat([
            new_df,
            train_data[train_data["Number Label"] == i].sample(value_no2_max, random_state=1),
        ])
    else:
        new_df = pd.concat([
            new_df,
            train_data[train_data["Number Label"] == i]
        ])

In [ ]:
new_df = new_df.reset_index(drop=True).drop(columns=["Unnamed: 0"])
new_df, ohe = f_p.ohe_hot_encoding(new_df, categorical_list=["Protocol", "Destination Port"])

binary = new_df
binary["Number Label"] = binary["Number Label"].apply(lambda x: 1 if x == 0 else 0)

binary.to_csv("./onehot_bi.csv")